# COBRApy

COBRApy is a package for constraint-based modeling of biological networks written in Python.

This tool allows loading and inspecting Genome-Scale Metabolic (GEM) models written in the Systems Biology Markup Language (SBML) format.

Using COBRApy, one can analyze the following model contents:
1. Reactions
2. Metabolites
3. Genes
4. Exchange reactions (Environmental Conditions)

COBRApy allows manipulating the contents of a GEM model. For instance, one can edit reactions' flux bounds, knock out a metabolic gene, or change environmental conditions.

COBRApy contains flux analysis methods to simulate an organism's phenotypic behavior. These include Flux Balance Analysis (FBA), Parsimonious FBA, or Flux Variability Analysis (FVA).

The simulation of gene and reaction deletions for a given GEM model is a straightforward process. One can simulate single or double knockouts using one of the flux analysis methods mentioned above.

## Installation


### Requirements
The following requirements are needed to use COBRApy:
- Python 3.6 or higher
- pip must be installed
- GLPK is the default solver, but CPLEX is preferred


### How to install COBRApy?
```
pip install cobra
```

# Exercise 6 - Phenotype prediction

## Working with a GEM model

For this practical session, we will be using the following model:
- _iMM904_ model which contains the whole metabolism of _S. cerevisiae_ -> file: ../data/iMM904.xml

You can read more about _S. cerevisiae_ iMM804 model (Mo et al., 2009) in the following links:
- https://bmcsystbiol.biomedcentral.com/articles/10.1186/1752-0509-3-37
- http://bigg.ucsd.edu/models/iMM904

This exercise consists of exploring the phenotype prediction tools of COBRApy. Thus, the following steps will be followed:
- Perform an pFBA simulation using an aerobic/anaerobic conditions;
- Perform an FVA simulation;
- Perform gene deletions;
- Perform single deletions
- Perform production envelopes

In [ ]:
# imports
import cobra
import escher

In [ ]:
# Loading a model
model_path = '../data/iMM904.xml'
model = cobra.io.read_sbml_model(model_path)

model

## Phenotype Prediction

### pFBA - Aerobiosis

In [ ]:
#check model exchanges
for exchange in model.exchanges:
    print(exchange.name, '->', exchange.bounds)

In [ ]:
model.exchanges.EX_o2_e

In [ ]:
#performing pfba simulation
pfba_solution = cobra.flux_analysis.pfba(model)
model.summary(pfba_solution)

### Setting up anaerobiosis

In [ ]:
# setting the environmental conditions to replicate an anaerobic medium
anaerobic_conditions = {'EX_ergstest_SC_e': (-2, 1000),
                        'EX_pro__L_e': (-2, 1000), 
                        'EX_ptd1ino_SC_e':(-2, 1000), 
                        'EX_zymst_e': (-2, 1000),
                        'EX_o2_e': (0, 1000)}

for exchange_id, bds in anaerobic_conditions.items():
    model.exchanges.get_by_id(exchange_id).bounds = bds

### pFBA - Anaerobiosis

In [ ]:
#performing pfba simulation
pfba_solution = cobra.flux_analysis.pfba(model)
model.summary(pfba_solution)

### Flux Variability Analysis (FVA)

In [ ]:
#performing fva simulation
fva_solution = cobra.flux_analysis.flux_variability_analysis(model, fraction_of_optimum=0.1)
fva_solution

In [ ]:
# maximum theoretical production rates of Ethanol (EX_eth_e), Carbon dioxide (EX_co2_e)

print(f'Theoretical production rate interval for Ethanol:', fva_solution.loc['EX_etoh_e', 'minimum'], 'to', fva_solution.loc['EX_etoh_e', 'maximum'], 'mmol/gDW/h')
print(f'Theoretical production rate interval for Carbon Dioxide:', fva_solution.loc['EX_co2_e', 'minimum'], 'to', fva_solution.loc['EX_co2_e', 'maximum'], 'mmol/gDW/h')

### Simulating Deletions

#### Knock outs

In [ ]:
model.genes.YPL262W

In [ ]:
model.reactions.FUM

In [ ]:
model.reactions.FUMm

In [ ]:
#knock out the YPL262W (FUM1) gene associated with FUM and FUMm reactions
with model:
    model.genes.YPL262W.knock_out()
    pfba_solution = cobra.flux_analysis.pfba(model)
    print('FUM1 mutant growth rate: ', pfba_solution.fluxes['BIOMASS_SC5_notrace'])
    print('Ethanol production rate: ', pfba_solution.fluxes['EX_etoh_e'])

In [ ]:
builder = escher.Builder(map_name='iMM904.Central carbon metabolism', model=model, reaction_data=pfba_solution.fluxes)
builder

#### Single Deletions

In [ ]:
#single reaction deletion
double_reaction_deletion_results = cobra.flux_analysis.single_reaction_deletion(model)
double_reaction_deletion_results

In [ ]:
#single gene deletion
single_gene_deletion_results = cobra.flux_analysis.single_gene_deletion(model)
single_gene_deletion_results

### Production envelopes

In [ ]:
# setting the environmental conditions to replicate an aerobic medium again
anaerobic_conditions = {'EX_ergstest_SC_e': (0, 1000),
                        'EX_pro__L_e': (0, 1000), 
                        'EX_ptd1ino_SC_e':(0, 1000), 
                        'EX_zymst_e': (0, 1000),
                        'EX_o2_e': (-60, 1000)}

for exchange_id, bds in anaerobic_conditions.items():
    model.exchanges.get_by_id(exchange_id).bounds = bds

In [ ]:
# Production envelope for Ethanol production as a function of oxygen consumption rate
prod_env = cobra.flux_analysis.production_envelope(model, ["EX_o2_e"], objective="EX_etoh_e", carbon_sources="EX_glc__D_e")
prod_env.plot(kind='line', x='EX_o2_e', y='carbon_yield_maximum')